Steps:

- Filter protein dataset
    - Organisms
- Annotate with GO terms
- Create subgraph of GO
    - transmembrane transporter activity
    - only go terms that occur in protein dataset
- Annotate GO network with ChEBI terms
- Chebi Network: Sub-Network of molecules that occur in organism
    - Node size: Number of proteins
    - Edges: Overlap in terms of proteins

TODO compare to graph notebook

TODO turn into functions in graph.py

## Protein dataset

In [4]:
from subpred.util import load_df
df_uniprot = load_df("uniprot")
df_uniprot

,gene_names,protein_names,reviewed,protein_existence,sequence,organism_id
Uniprot,,,,,,
A0A0C5B5G6,MT-RNR1,Mitochondrial-derived peptide MOTS-c (Mitochon...,True,1,MRWQEMGYIFYPRKLR,9606
A0A1B0GTW7,CIROP LMLN2,Ciliated left-right organizer metallopeptidase...,True,1,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...,9606
A0JNW5,BLTP3B KIAA0701 SHIP164 UHRF1BP1L,Bridge-like lipid transfer protein family memb...,True,1,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,9606
A0JP26,POTEB3,POTE ankyrin domain family member B3,True,1,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,9606
A0PK11,CLRN2,Clarin-2,True,1,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,9606
...,...,...,...,...,...,...
X5L4R4,NOD-2,Nucleotide-binding oligomerization domain-cont...,False,2,MSPGCYKGWPFNCHLSHEEDKRRNETLLQEAETSNLQITASFVSGL...,586796
X5MBL2,GT34D,"Putative galacto(Gluco)mannan alpha-1,6-galact...",False,2,KVLYDRAFNSSDDQSALVYLLLKEKDKWADRIFIEHKYYLNGYWLD...,3352
X5MFI4,GT34D,"Putative galacto(Gluco)mannan alpha-1,6-galact...",False,2,MDEDVLCKGPLHGGSARSLKGSLKRLKRIMESLNDGLIFMGGAVSA...,3352


In [5]:
ORGANISM_ID = 83333

df_uniprot_organism = df_uniprot[df_uniprot.organism_id == ORGANISM_ID].drop(
    "organism_id", axis=1
)
df_uniprot_organism.shape[0]

3284

In [6]:
df_uniprot_organism = df_uniprot_organism[
    ~df_uniprot_organism.gene_names.isnull()
    & ~df_uniprot_organism.protein_names.isnull()
    & ~df_uniprot_organism.sequence.isnull()
]
df_uniprot_organism.shape[0]

3284

In [7]:
REVIEWED_ONLY = True
PROTEIN_EXISTENCE_PROTEIN_LEVEL = True

display(
    df_uniprot_organism[["reviewed", "protein_existence"]]
    .groupby(["reviewed", "protein_existence"])
    .apply(len)
)

if REVIEWED_ONLY:
    df_uniprot_organism = df_uniprot_organism[df_uniprot_organism.reviewed]
if PROTEIN_EXISTENCE_PROTEIN_LEVEL:
    df_uniprot_organism = df_uniprot_organism[
        df_uniprot_organism.protein_existence == 1
    ]

reviewed  protein_existence
False     1                       1
          2                       1
True      1                    3118
          2                     164
dtype: int64

## GO Annotations

In [8]:
df_goa_uniprot = load_df("go")
df_goa_uniprot

,Uniprot,qualifier,go_id,evidence_code,aspect
0,A0A009FND8,enables,GO:0000166,IEA,F
1,A0A009FND8,enables,GO:0005524,IEA,F
2,A0A009FND8,enables,GO:0051082,IEA,F
3,A0A009FND8,enables,GO:0140662,IEA,F
4,A0A009FND8,involved_in,GO:0006457,IEA,P
...,...,...,...,...,...
7452013,Z9JND5,enables,GO:0000166,IEA,F
7452014,Z9JND5,enables,GO:0005524,IEA,F
7452015,Z9JND5,enables,GO:0051082,IEA,F
7452016,Z9JND5,enables,GO:0140662,IEA,F


Filter GO annotations for the organism:

In [9]:
df_goa_uniprot_organism = df_goa_uniprot[
    df_goa_uniprot.Uniprot.isin(df_uniprot_organism.index)
].reset_index(drop=True)

df_goa_uniprot_organism

,Uniprot,qualifier,go_id,evidence_code,aspect
0,A5A615,located_in,GO:0005886,IDA,C
1,A5A615,located_in,GO:0005886,IEA,C
2,A5A615,located_in,GO:0005886,RCA,C
3,A5A615,located_in,GO:0016020,IEA,C
4,A5A616,acts_upstream_of_or_within,GO:0010350,IEP,P
...,...,...,...,...,...
39156,Q93K97,involved_in,GO:0006753,IBA,P
39157,Q93K97,involved_in,GO:0019693,IBA,P
39158,Q93K97,is_active_in,GO:0005829,IBA,C
39159,Q93K97,located_in,GO:0005829,HDA,C


In [21]:
# TODO ancestors
# TODO IEA, qualifiers?

## GO NetworkX

In [11]:
import obonet
import networkx as nx

graph_go = obonet.read_obo("../data/raw/ontologies/go.obo", ignore_obsolete=True)

Dicts for converting between labels and identifiers:

In [ ]:
id_to_name = {id: data["name"] for id, data in graph_go.nodes(data=True)}
name_to_id = {name: id for id, name in id_to_name.items()}

Are there any GO Uniprot annotations that are not included in the graph? If not, the two files might be based on two different versions of Uniprot, and should be updated to the same version.

In [13]:
df_goa_uniprot_organism[~df_goa_uniprot_organism.go_id.isin(set(graph_go.nodes()))].shape[0]
# TODO filter for that?

0

What edge-annotations are in the data?

In [17]:
{edge[2] for edge in graph_go.edges(data=True, keys=True)}

{'ends_during',
 'happens_during',
 'has_part',
 'is_a',
 'negatively_regulates',
 'occurs_in',
 'part_of',
 'positively_regulates',
 'regulates'}

Creating sub-graph for Molecular Function terms:

In [40]:
graph_go_mf = graph_go.subgraph(
    nodes=[
        node
        for node, data in graph_go.nodes(data=True)
        if data["namespace"] == "molecular_function"
    ]
)

Which relations can we find in the MF-subgraph?

- has_part, part_of: Used for example for protein complexes and their complex members
- regulation: For transcription factors, messengers, etc.
- is_a: Direct, logical relation. This is what we want for finding all sub-types of transmembrane transporter activity.

In [41]:
{edge[2] for edge in graph_go_mf.edges(data=True, keys=True)}

{'has_part',
 'is_a',
 'negatively_regulates',
 'part_of',
 'positively_regulates',
 'regulates'}

In [47]:
# edge subgraph only contains nodes that are attached to the specified edges, so that should be fine.
graph_go_mf_isa = graph_go_mf.edge_subgraph({edge for edge in graph_go_mf.edges(keys=True) if edge[2] == "is_a"})

13736

The subgraph should be acyclic, is it?

In [49]:
nx.is_directed_acyclic_graph(graph_go_mf_isa)

True

13985

## Add abstract GO terms to annotation data

In [50]:
# TODO filter network for e coli proteins

## Annotate GO network with number of organism annotations

## Link from GO to ChEBI

ChEBI NetworkX